# Create an initial dataset with elements from COCO dataset 
# and elements from densepose dataset condensing left-right in only one class
## all body parts included

## corresponds with version 14


# Getting data and looking into the structure

In [1]:
import cv2
import json
from json import JSONEncoder
import pandas as pd
import numpy as np
import pycocotools.mask as mask_util

from tqdm import tqdm

In [2]:
# version number of the json files
version = 1
# datapath = '/home/ubuntu/dormakaba/coco'
datapath = '/mnt/DATASSD/DataSandbox/coco'

In [3]:
with open(f'{datapath}/annotations/instances_val2014.json') as json_file:
    data = json.load(json_file)

In [4]:
with open(f'{datapath}/annotations/densepose_coco_2014_valminusminival.json') as json_file:
    person_data = json.load(json_file)

In [5]:
data_df = pd.json_normalize(data)
data_df.keys()

Index(['images', 'licenses', 'annotations', 'categories', 'info.description',
       'info.url', 'info.version', 'info.year', 'info.contributor',
       'info.date_created'],
      dtype='object')

In [6]:
person_data_df = pd.json_normalize(person_data)
person_data_df.keys()

Index(['images', 'annotations', 'categories'], dtype='object')

In [7]:
person_data_df['categories'].iloc[0][0]

{'supercategory': 'person',
 'id': 1,
 'keypoints': ['nose',
  'left_eye',
  'right_eye',
  'left_ear',
  'right_ear',
  'left_shoulder',
  'right_shoulder',
  'left_elbow',
  'right_elbow',
  'left_wrist',
  'right_wrist',
  'left_hip',
  'right_hip',
  'left_knee',
  'right_knee',
  'left_ankle',
  'right_ankle'],
 'name': 'person',
 'skeleton': [[16, 14],
  [14, 12],
  [17, 15],
  [15, 13],
  [12, 13],
  [6, 12],
  [7, 13],
  [6, 7],
  [6, 8],
  [7, 9],
  [8, 10],
  [9, 11],
  [2, 3],
  [1, 2],
  [1, 3],
  [2, 4],
  [3, 5],
  [4, 6],
  [5, 7]]}

In [8]:
data_df['categories'][0]

[{'supercategory': 'person', 'id': 1, 'name': 'person'},
 {'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'},
 {'supercategory': 'vehicle', 'id': 3, 'name': 'car'},
 {'supercategory': 'vehicle', 'id': 4, 'name': 'motorcycle'},
 {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'},
 {'supercategory': 'vehicle', 'id': 6, 'name': 'bus'},
 {'supercategory': 'vehicle', 'id': 7, 'name': 'train'},
 {'supercategory': 'vehicle', 'id': 8, 'name': 'truck'},
 {'supercategory': 'vehicle', 'id': 9, 'name': 'boat'},
 {'supercategory': 'outdoor', 'id': 10, 'name': 'traffic light'},
 {'supercategory': 'outdoor', 'id': 11, 'name': 'fire hydrant'},
 {'supercategory': 'outdoor', 'id': 13, 'name': 'stop sign'},
 {'supercategory': 'outdoor', 'id': 14, 'name': 'parking meter'},
 {'supercategory': 'outdoor', 'id': 15, 'name': 'bench'},
 {'supercategory': 'animal', 'id': 16, 'name': 'bird'},
 {'supercategory': 'animal', 'id': 17, 'name': 'cat'},
 {'supercategory': 'animal', 'id': 18, 'name': 'dog'},

In [9]:
person_data_df['annotations'].iloc[0][0]

{'segmentation': [[217.17,
   355,
   218.61,
   349.24,
   212.85,
   324.79,
   214.29,
   313.29,
   225.8,
   296.03,
   244.49,
   296.03,
   256,
   304.66,
   257.44,
   339.18,
   257.44,
   350.68,
   325.03,
   382.32,
   352.36,
   406.77,
   345.17,
   418.28,
   343.73,
   422.59,
   332.22,
   422.59,
   312.09,
   409.65,
   279.01,
   396.7,
   273.26,
   389.51,
   264.63,
   445.6,
   258.88,
   534.77,
   251.69,
   563.54,
   240.18,
   580.79,
   244.49,
   587.98,
   224.36,
   587.98,
   208.54,
   587.98,
   207.1,
   587.98,
   202.79,
   582.23,
   201.35,
   562.1,
   201.35,
   541.96,
   201.35,
   501.69,
   202.79,
   390.95,
   201.35,
   382.32,
   178.34,
   393.83,
   171.15,
   393.83,
   161.08,
   390.95,
   145.26,
   383.76,
   135.19,
   375.13,
   116.49,
   369.38,
   109.3,
   369.38,
   117.93,
   362.19,
   136.63,
   359.31,
   142.38,
   356.43,
   143.82,
   356.43,
   169.71,
   370.82,
   199.91,
   359.31]],
 'num_keypoints': 17,
 'dp

In [10]:
data_df['annotations'].iloc[0][0]

{'segmentation': [[239.97,
   260.24,
   222.04,
   270.49,
   199.84,
   253.41,
   213.5,
   227.79,
   259.62,
   200.46,
   274.13,
   202.17,
   277.55,
   210.71,
   249.37,
   253.41,
   237.41,
   264.51,
   242.54,
   261.95,
   228.87,
   271.34]],
 'area': 2765.1486500000005,
 'iscrowd': 0,
 'image_id': 558840,
 'bbox': [199.84, 200.46, 77.71, 70.88],
 'category_id': 58,
 'id': 156}

# Generating classes

In [11]:
COCO_CLASSES = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
                'train', 'truck', 'boat', 'traffic light', 'fire hydrant',
                'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog',
                'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe',
                'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
                'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat',
                'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
                'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
                'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot',
                'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
                'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop',
                'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven',
                'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
                'scissors', 'teddy bear', 'hair drier', 'toothbrush']
                
# THE NEW CUSTOM CLASSES CONDENSE THE LEFT-RIGHT IN ONLY ONE CAT
CUSTOM_CLASSES = ['person', 'bicycle', 'cat', 'dog', 'backpack', 'umbrella', 'handbag', 'suitcase', 
                'sports ball', 'baseball bat', 'skateboard', 'tennis racket', 'bottle', 'wine glass', 
                'sandwich', 'chair', 'laptop', 'cell phone', 'book', 'clock', 'scissors', 'teddy bear',
                'torso', 'hand', 'foot', 'upleg', 'lowleg', 'uparm', 'lowarm', 'head']

# THE NEW SMALL CUSTOM CLASSES CONDENSE THE LEFT-RIGHT IN ONLY ONE CAT
SMALL_CUSTOM_CLASSES = ['bicycle', 'cat', 'dog', 'backpack', 'handbag', 'suitcase', 
                        'foot', 'upleg', 'lowleg']
            
CATEGORIES = [{'supercategory': 'person', 'id': 1, 'name': 'person'},
            {'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'},
            {'supercategory': 'vehicle', 'id': 3, 'name': 'car'},
            {'supercategory': 'vehicle', 'id': 4, 'name': 'motorcycle'},
            {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'},
            {'supercategory': 'vehicle', 'id': 6, 'name': 'bus'},
            {'supercategory': 'vehicle', 'id': 7, 'name': 'train'},
            {'supercategory': 'vehicle', 'id': 8, 'name': 'truck'},
            {'supercategory': 'vehicle', 'id': 9, 'name': 'boat'},
            {'supercategory': 'outdoor', 'id': 10, 'name': 'traffic light'},
            {'supercategory': 'outdoor', 'id': 11, 'name': 'fire hydrant'},
            {'supercategory': 'outdoor', 'id': 13, 'name': 'stop sign'},
            {'supercategory': 'outdoor', 'id': 14, 'name': 'parking meter'},
            {'supercategory': 'outdoor', 'id': 15, 'name': 'bench'},
            {'supercategory': 'animal', 'id': 16, 'name': 'bird'},
            {'supercategory': 'animal', 'id': 17, 'name': 'cat'},
            {'supercategory': 'animal', 'id': 18, 'name': 'dog'},
            {'supercategory': 'animal', 'id': 19, 'name': 'horse'},
            {'supercategory': 'animal', 'id': 20, 'name': 'sheep'},
            {'supercategory': 'animal', 'id': 21, 'name': 'cow'},
            {'supercategory': 'animal', 'id': 22, 'name': 'elephant'},
            {'supercategory': 'animal', 'id': 23, 'name': 'bear'},
            {'supercategory': 'animal', 'id': 24, 'name': 'zebra'},
            {'supercategory': 'animal', 'id': 25, 'name': 'giraffe'},
            {'supercategory': 'accessory', 'id': 27, 'name': 'backpack'},
            {'supercategory': 'accessory', 'id': 28, 'name': 'umbrella'},
            {'supercategory': 'accessory', 'id': 31, 'name': 'handbag'},
            {'supercategory': 'accessory', 'id': 32, 'name': 'tie'},
            {'supercategory': 'accessory', 'id': 33, 'name': 'suitcase'},
            {'supercategory': 'sports', 'id': 34, 'name': 'frisbee'},
            {'supercategory': 'sports', 'id': 35, 'name': 'skis'},
            {'supercategory': 'sports', 'id': 36, 'name': 'snowboard'},
            {'supercategory': 'sports', 'id': 37, 'name': 'sports ball'},
            {'supercategory': 'sports', 'id': 38, 'name': 'kite'},
            {'supercategory': 'sports', 'id': 39, 'name': 'baseball bat'},
            {'supercategory': 'sports', 'id': 40, 'name': 'baseball glove'},
            {'supercategory': 'sports', 'id': 41, 'name': 'skateboard'},
            {'supercategory': 'sports', 'id': 42, 'name': 'surfboard'},
            {'supercategory': 'sports', 'id': 43, 'name': 'tennis racket'},
            {'supercategory': 'kitchen', 'id': 44, 'name': 'bottle'},
            {'supercategory': 'kitchen', 'id': 46, 'name': 'wine glass'},
            {'supercategory': 'kitchen', 'id': 47, 'name': 'cup'},
            {'supercategory': 'kitchen', 'id': 48, 'name': 'fork'},
            {'supercategory': 'kitchen', 'id': 49, 'name': 'knife'},
            {'supercategory': 'kitchen', 'id': 50, 'name': 'spoon'},
            {'supercategory': 'kitchen', 'id': 51, 'name': 'bowl'},
            {'supercategory': 'food', 'id': 52, 'name': 'banana'},
            {'supercategory': 'food', 'id': 53, 'name': 'apple'},
            {'supercategory': 'food', 'id': 54, 'name': 'sandwich'},
            {'supercategory': 'food', 'id': 55, 'name': 'orange'},
            {'supercategory': 'food', 'id': 56, 'name': 'broccoli'},
            {'supercategory': 'food', 'id': 57, 'name': 'carrot'},
            {'supercategory': 'food', 'id': 58, 'name': 'hot dog'},
            {'supercategory': 'food', 'id': 59, 'name': 'pizza'},
            {'supercategory': 'food', 'id': 60, 'name': 'donut'},
            {'supercategory': 'food', 'id': 61, 'name': 'cake'},
            {'supercategory': 'furniture', 'id': 62, 'name': 'chair'},
            {'supercategory': 'furniture', 'id': 63, 'name': 'couch'},
            {'supercategory': 'furniture', 'id': 64, 'name': 'potted plant'},
            {'supercategory': 'furniture', 'id': 65, 'name': 'bed'},
            {'supercategory': 'furniture', 'id': 67, 'name': 'dining table'},
            {'supercategory': 'furniture', 'id': 70, 'name': 'toilet'},
            {'supercategory': 'electronic', 'id': 72, 'name': 'tv'},
            {'supercategory': 'electronic', 'id': 73, 'name': 'laptop'},
            {'supercategory': 'electronic', 'id': 74, 'name': 'mouse'},
            {'supercategory': 'electronic', 'id': 75, 'name': 'remote'},
            {'supercategory': 'electronic', 'id': 76, 'name': 'keyboard'},
            {'supercategory': 'electronic', 'id': 77, 'name': 'cell phone'},
            {'supercategory': 'appliance', 'id': 78, 'name': 'microwave'},
            {'supercategory': 'appliance', 'id': 79, 'name': 'oven'},
            {'supercategory': 'appliance', 'id': 80, 'name': 'toaster'},
            {'supercategory': 'appliance', 'id': 81, 'name': 'sink'},
            {'supercategory': 'appliance', 'id': 82, 'name': 'refrigerator'},
            {'supercategory': 'indoor', 'id': 84, 'name': 'book'},
            {'supercategory': 'indoor', 'id': 85, 'name': 'clock'},
            {'supercategory': 'indoor', 'id': 86, 'name': 'vase'},
            {'supercategory': 'indoor', 'id': 87, 'name': 'scissors'},
            {'supercategory': 'indoor', 'id': 88, 'name': 'teddy bear'},
            {'supercategory': 'indoor', 'id': 89, 'name': 'hair drier'},
            {'supercategory': 'indoor', 'id': 90, 'name': 'toothbrush'}]

FROM_COCO_CLASSES = ['person', 'bicycle', 'cat', 'dog', 'backpack', 'umbrella', 'handbag', 'suitcase', 
                'sports ball', 'baseball bat', 'skateboard', 'tennis racket', 'bottle', 'wine glass', 
                'sandwich', 'chair', 'laptop', 'cell phone', 'book', 'clock', 'scissors', 'teddy bear']

EXTENDED_DENSEPOSE_CLASSES = ['torso', 'hand1', 'hand2', 
                    'foot1', 'foot2', 'upleg1', 'upleg2', 
                    'lowleg1', 'lowleg2','uparm1', 'uparm2', 
                    'lowarm1','lowarm2', 'head']

DENSEPOSE_CLASSES = ['torso', 'hand',  
                        'foot', 'upleg', 
                        'lowleg', 'uparm', 
                        'lowarm', 'head']

# realates image ID with category
COCO_LABEL_MAP = {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8,
                  9: 9, 10: 10, 11: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16,
                  18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24,
                  27: 25, 28: 26, 31: 27, 32: 28, 33: 29, 34: 30, 35: 31, 36: 32,
                  37: 33, 38: 34, 39: 35, 40: 36, 41: 37, 42: 38, 43: 39, 44: 40,
                  46: 41, 47: 42, 48: 43, 49: 44, 50: 45, 51: 46, 52: 47, 53: 48,
                  54: 49, 55: 50, 56: 51, 57: 52, 58: 53, 59: 54, 60: 55, 61: 56,
                  62: 57, 63: 58, 64: 59, 65: 60, 67: 61, 70: 62, 72: 63, 73: 64,
                  74: 65, 75: 66, 76: 67, 77: 68, 78: 69, 79: 70, 80: 71, 81: 72,
                  82: 73, 84: 74, 85: 75, 86: 76, 87: 77, 88: 78, 89: 79, 90: 80}

# this map relates body part position to unique identifier (avoiding left-right problem)
PERSON_LABEL_MAP = {0:0, 1: 1, 2: 1, 
                    3: 2, 4: 2, 5: 3, 6: 3, 
                    7: 4, 8: 4, 9: 5, 10: 5, 
                    11: 6, 12: 6, 13: 7}

In [12]:
REVERSE_COCO_LABEL_MAP = {}
for key, value in COCO_LABEL_MAP.items():
    REVERSE_COCO_LABEL_MAP[value] = key

# working with categories

In [13]:
# creates the categories list from COCO selected classes
custom_categories = []
for cat_data in data_df['categories'][0]:
    if cat_data['name'] in SMALL_CUSTOM_CLASSES:
        custom_categories.append(cat_data)
custom_categories

[{'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'},
 {'supercategory': 'animal', 'id': 17, 'name': 'cat'},
 {'supercategory': 'animal', 'id': 18, 'name': 'dog'},
 {'supercategory': 'accessory', 'id': 27, 'name': 'backpack'},
 {'supercategory': 'accessory', 'id': 31, 'name': 'handbag'},
 {'supercategory': 'accessory', 'id': 33, 'name': 'suitcase'}]

In [14]:
# Adds hupan parts selected classes
i = 91
for nc in DENSEPOSE_CLASSES:
    if nc in SMALL_CUSTOM_CLASSES:
        c = {'supercategory': 'person', 'id': i, 'name': nc}
        custom_categories.append(c)
        i +=1
custom_categories

[{'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'},
 {'supercategory': 'animal', 'id': 17, 'name': 'cat'},
 {'supercategory': 'animal', 'id': 18, 'name': 'dog'},
 {'supercategory': 'accessory', 'id': 27, 'name': 'backpack'},
 {'supercategory': 'accessory', 'id': 31, 'name': 'handbag'},
 {'supercategory': 'accessory', 'id': 33, 'name': 'suitcase'},
 {'supercategory': 'person', 'id': 91, 'name': 'foot'},
 {'supercategory': 'person', 'id': 92, 'name': 'upleg'},
 {'supercategory': 'person', 'id': 93, 'name': 'lowleg'}]

In [15]:
# this is the list of the new categories
print(custom_categories)
print()
print(len(SMALL_CUSTOM_CLASSES), "-", len(custom_categories))

cc_list = [0]*(len(SMALL_CUSTOM_CLASSES))
CUSTOM_LABEL_MAP = {}
CUSTOM_ID_MAP = {}

for i, cc in enumerate(custom_categories):
    CUSTOM_ID_MAP[i] = cc['id']
    CUSTOM_LABEL_MAP[i] = cc['name']
    cc_list[i] = cc['name']

# the list of the names of the categories
print(CUSTOM_ID_MAP)
print(CUSTOM_LABEL_MAP)
print(cc_list)

[{'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'}, {'supercategory': 'animal', 'id': 17, 'name': 'cat'}, {'supercategory': 'animal', 'id': 18, 'name': 'dog'}, {'supercategory': 'accessory', 'id': 27, 'name': 'backpack'}, {'supercategory': 'accessory', 'id': 31, 'name': 'handbag'}, {'supercategory': 'accessory', 'id': 33, 'name': 'suitcase'}, {'supercategory': 'person', 'id': 91, 'name': 'foot'}, {'supercategory': 'person', 'id': 92, 'name': 'upleg'}, {'supercategory': 'person', 'id': 93, 'name': 'lowleg'}]

9 - 9
{0: 2, 1: 17, 2: 18, 3: 27, 4: 31, 5: 33, 6: 91, 7: 92, 8: 93}
{0: 'bicycle', 1: 'cat', 2: 'dog', 3: 'backpack', 4: 'handbag', 5: 'suitcase', 6: 'foot', 7: 'upleg', 8: 'lowleg'}
['bicycle', 'cat', 'dog', 'backpack', 'handbag', 'suitcase', 'foot', 'upleg', 'lowleg']


In [16]:
coco_category_id_list = [0]*len(cc_list)
for cat in data_df['categories'][0]:
    if cat['name'] in cc_list:
        print(cat, cat['name'], cat['id'])
        pos = cc_list.index(cat['name'])
        coco_category_id_list[pos] = cat['id']

# this is the list of the COCO ids of the new categories (when exists)
print(coco_category_id_list)
print(cc_list)

{'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'} bicycle 2
{'supercategory': 'animal', 'id': 17, 'name': 'cat'} cat 17
{'supercategory': 'animal', 'id': 18, 'name': 'dog'} dog 18
{'supercategory': 'accessory', 'id': 27, 'name': 'backpack'} backpack 27
{'supercategory': 'accessory', 'id': 31, 'name': 'handbag'} handbag 31
{'supercategory': 'accessory', 'id': 33, 'name': 'suitcase'} suitcase 33
[2, 17, 18, 27, 31, 33, 0, 0, 0]
['bicycle', 'cat', 'dog', 'backpack', 'handbag', 'suitcase', 'foot', 'upleg', 'lowleg']


In [17]:
# this variable has the coco category id in the position of the custom category
custom_category_coco_category = [0]*len(cc_list)
for i, cat_name in enumerate(COCO_CLASSES):
    if cat_name in cc_list:
        cc_pos = cc_list.index(cat_name)
        coco_cat_id = REVERSE_COCO_LABEL_MAP[i+1]
        custom_category_coco_category[cc_pos] = coco_cat_id
        # print(i, cat_name, cc_pos, coco_cat_id)

print(custom_category_coco_category)

[2, 17, 18, 27, 31, 33, 0, 0, 0]


# looking at images

In [18]:
new_person_images = person_data_df['images'][0]
print( len(new_person_images), new_person_images[0].keys())

5984 dict_keys(['license', 'file_name', 'coco_url', 'height', 'width', 'date_captured', 'flickr_url', 'id'])


In [19]:
images_person_data_df = pd.DataFrame(person_data_df['images'][0])

In [20]:
images_person_data_df.head(2)

,license,file_name,coco_url,height,width,date_captured,flickr_url,id
0,1,COCO_val2014_000000109229.jpg,http://mscoco.org/images/109229,640,514,2013-11-18 17:27:34,http://farm3.staticflickr.com/2342/1794334693_...,109229
1,1,COCO_val2014_000000393243.jpg,http://mscoco.org/images/393243,640,480,2013-11-20 13:20:35,http://farm3.staticflickr.com/2294/1562760533_...,393243


In [21]:
person_data_df['images'][0][0]

{'license': 1,
 'file_name': 'COCO_val2014_000000109229.jpg',
 'coco_url': 'http://mscoco.org/images/109229',
 'height': 640,
 'width': 514,
 'date_captured': '2013-11-18 17:27:34',
 'flickr_url': 'http://farm3.staticflickr.com/2342/1794334693_c3e27b61dd_z.jpg',
 'id': 109229}

# looking at images
## Those are the images I'll use in the dataset that come from 
## the DENSEPOSE dataset

In [22]:
# list of image ids from human parts data
# from here we will use all images
new_person_images_list = [a['id'] for a in person_data_df['images'][0]]
len(new_person_images_list)

5984

In [23]:
# list of img ids from coco dataset
# that have a cat id in the list of coco selected categories
# removing duplicates
all_images_list = set([a['image_id'] for a in data_df['annotations'][0] if a['category_id'] in coco_category_id_list])
len(all_images_list)

7340

In [24]:
images_from_coco_not_in_dense = [img for img in all_images_list if img not in new_person_images_list]
len(images_from_coco_not_in_dense)

6061

In [25]:
images_from_dense_not_in_custom = [img for img in new_person_images_list if img not in all_images_list]
len(images_from_dense_not_in_custom)

4705

In [26]:
len(data_df['images'][0])

40504

# annotation keys

In [27]:
# person annotations
# extraer segmentation: anotaciòn individual de cada persona detectada
# extrater dp_masks: e insertar una anotación por cada uno de los 14 elementos de la persona que tienen anotaciones
# si se puede, transformar el mapa de bits en 
print(person_data_df['annotations'][0][0].keys())
print(person_data_df['annotations'][0][0]['dp_masks'])

dict_keys(['segmentation', 'num_keypoints', 'dp_masks', 'area', 'dp_I', 'iscrowd', 'keypoints', 'dp_x', 'dp_U', 'image_id', 'dp_V', 'bbox', 'category_id', 'dp_y', 'id'])
[{'counts': '[bh03m70O5I8I7B=D=K=C;E0000001N101O1N101N101O000010O010O1O01O010O010O001O010O1O010N1000O100O001O100O1O01O0100O010OO12O000000001O000000001O000O12NJ7D;ZNPJR1NjI8l6Hf0O2O0O2O0000XVc0', 'size': [256, 256]}, {'counts': 'mQ11o72N000O1000001N0100000O100O1O0100000O100O10O10010O01O1O001OO1O2N1N3M2O10Wfe1', 'size': [256, 256]}, {'counts': 'Pkj1173]7:N0O3N1N2N010O02O0O2N101M2M5LQU1', 'size': [256, 256]}, {'counts': 'ZgQ16g77L3M1O001O001OO10000000O14L5K4L4LYhi0', 'size': [256, 256]}, {'counts': 'j_l01o72N2N1N1000000000000000000O1000000O12N3M3MQPn0', 'size': [256, 256]}, {'counts': 'U\\j0h1S64N21O101N1O2N2O001O001O00001O001O000010O100O100O101NN3ROn0I6J7Ikcm0', 'size': [256, 256]}, {'counts': 'P]Q1a0Y7d0\\O5K6M210O10O01O002O0O2N101N101N103L3MN3N2N2M3N200N2I7F:I7K5J6K4IXTf0', 'size': [256, 256]}, {'counts': 'm]j09f7b0^O<

In [28]:
print(person_data_df['annotations'][0][0])

{'segmentation': [[217.17, 355, 218.61, 349.24, 212.85, 324.79, 214.29, 313.29, 225.8, 296.03, 244.49, 296.03, 256, 304.66, 257.44, 339.18, 257.44, 350.68, 325.03, 382.32, 352.36, 406.77, 345.17, 418.28, 343.73, 422.59, 332.22, 422.59, 312.09, 409.65, 279.01, 396.7, 273.26, 389.51, 264.63, 445.6, 258.88, 534.77, 251.69, 563.54, 240.18, 580.79, 244.49, 587.98, 224.36, 587.98, 208.54, 587.98, 207.1, 587.98, 202.79, 582.23, 201.35, 562.1, 201.35, 541.96, 201.35, 501.69, 202.79, 390.95, 201.35, 382.32, 178.34, 393.83, 171.15, 393.83, 161.08, 390.95, 145.26, 383.76, 135.19, 375.13, 116.49, 369.38, 109.3, 369.38, 117.93, 362.19, 136.63, 359.31, 142.38, 356.43, 143.82, 356.43, 169.71, 370.82, 199.91, 359.31]], 'num_keypoints': 17, 'dp_masks': [{'counts': '[bh03m70O5I8I7B=D=K=C;E0000001N101O1N101N101O000010O010O1O01O010O010O001O010O1O010N1000O100O001O100O1O01O0100O010OO12O000000001O000000001O000O12NJ7D;ZNPJR1NjI8l6Hf0O2O0O2O0000XVc0', 'size': [256, 256]}, {'counts': 'mQ11o72N000O1000001N010000

# generating annotations

In [29]:
def GetDensePoseMasks(Polys):
    mask_list = [0]*15
    for i in range(1,15):
        MaskGen = np.zeros([256,256])
        if(Polys[i-1]):
            current_mask = mask_util.decode(Polys[i-1])
            MaskGen[current_mask>0] = 1
        mask_list[i-1]=MaskGen
    return mask_list

In [30]:
def GetDensePoseMask(encoded_mask):
    MaskGen = np.zeros([256,256])
    current_mask = mask_util.decode(encoded_mask)
    MaskGen[current_mask>0] = 1
    return MaskGen

In [31]:
def get_poligons(mask):
    c = cv2.findContours(mask.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = [0]*2*len(c[0][0].tolist())
    for i,x in enumerate(c[0][0].tolist()):
        contours[2*i]=int(x[0][0])
        contours[2*i+1]= int(x[0][1])
    return contours

In [32]:
def CreateMaskBBox(mask, bbr, image_shape):
    final_mask = np.zeros([image_shape[1],image_shape[0]])
    x1 = int(bbr[0])
    y1 = int(bbr[1])
    x2 = int(bbr[0]+bbr[2])
    y2 = int(bbr[1]+bbr[3])
    x2 = min( [ x2, image_shape[0] ] )
    y2 = min( [ y2, image_shape[1] ] )
    # print(bbr, (x1,y1,x2,y2), (int(x2-x1),int(y2-y1)), mask.shape)
    resized_mask = cv2.resize( mask, (int(x2-x1),int(y2-y1)) ,interpolation=cv2.INTER_NEAREST)
    final_mask[y1:y2,x1:x2] = resized_mask
    az = np.nonzero(final_mask)
    bby1,bby2,bbx1,bbx2=np.min(az[0]), np.max(az[0]), np.min(az[1]), np.max(az[1])
    contours = get_poligons(final_mask.astype(np.uint8))
    encoded_final_mask = mask_util.encode(np.asfortranarray(final_mask.astype(np.uint8)))
    mask_area = int(mask_util.area(encoded_final_mask))
    # print((bbx1, bby1, bbx2-bbx1, bby2-bby1),encoded_final_mask)
    encoded_final_mask['counts'] = encoded_final_mask['counts'].decode("utf-8")
    return final_mask, (bbx1, bby1, bbx2-bbx1, bby2-bby1), mask_area, encoded_final_mask, contours

# create new annotations in coco style suitable to train YOLACT model

In [33]:
new_person_ann = []
for annotation in tqdm(person_data_df['annotations'][0]):
    # print(annotation['image_id'], annotation['bbox'])
    if annotation['iscrowd']==0 and annotation['image_id'] in new_person_images_list:
        ann_segment = annotation['segmentation']
        ann_bbox = [int(annotation['bbox'][0]),int(annotation['bbox'][1]),int(annotation['bbox'][2]),int(annotation['bbox'][3])]
        ann_area = annotation['area']
        ann_image_id = annotation['image_id']
        ann_id = annotation['id']
        ann_cat_id = 1 # only person cat so must be 1
        new_data = {'segmentation': ann_segment,
                    'area': ann_area,
                    'image_id': ann_image_id,
                    'id': ann_id,
                    'category_id': ann_cat_id,
                    'bbox': ann_bbox,
                    'iscrowd': 0}
        new_person_ann.append(new_data)
        a = images_person_data_df[images_person_data_df['id']==ann_image_id]
        image_size = [a['width'].values[0],a['height'].values[0]]

        if 'dp_masks' in annotation:
            for i, ann_mask in enumerate(annotation['dp_masks']):
                if len(ann_mask) > 1:
                    ann_mask_decoded = GetDensePoseMask(ann_mask)
                    mask, bbox, ann_area, encoded_mask, contours = CreateMaskBBox(ann_mask_decoded, annotation['bbox'], image_size )
                    ann_bbox = [int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3])]
                    ann_segment = encoded_mask
                    ann_image_id = annotation['image_id']
                    ann_id = -1
                    # for reusing ids I'll prepare a first approach 
                    # with a cat id dictionary
                    ann_cat_id = 91+PERSON_LABEL_MAP[i]
                    if len(contours)>4:
                        new_data = {'segmentation': [contours], # encoded_mask,
                                    'area': ann_area,
                                    'image_id': ann_image_id,
                                    'id': ann_id,
                                    'category_id': ann_cat_id,
                                    'bbox': ann_bbox,
                                    'iscrowd': 0}
                        new_person_ann.append(new_data)

100%|██████████| 25053/25053 [03:49<00:00, 109.39it/s]


In [34]:
xxx1 = [x for x in new_person_ann if x['category_id']==1]
xxx2 = [x for x in new_person_ann if x['category_id']!=1]
print(len(xxx1),len(xxx2))

24640 69369


In [35]:
# check that the list of annotated image_ids has the len len as densepose image id's
new_person_images_annotated_list = set([a['image_id'] for a in new_person_ann])
print( len(new_person_images_annotated_list), len(set(new_person_images_annotated_list)), len(set(new_person_images_list)))

5984 5984 5984


# till here I have all human data and parts segmented and annotated
# create new annotations from COCO dataset for the selected images

In [36]:
person_ann_list1 = [x for x in data_df['annotations'][0] if x['image_id'] in all_images_list and x['image_id'] not in new_person_images_list and x['category_id']==1 and x['iscrowd']==0]
person_ann_list2 = [x for x in data_df['annotations'][0] if x['image_id'] not in new_person_images_list and x['category_id'] in custom_category_coco_category and x['iscrowd']==0]
person_ann_list3 = [x for x in data_df['annotations'][0] if x['image_id'] in new_person_images_list and x['category_id']==1 and x['iscrowd']==0]
print(len(data_df['annotations'][0]), len(person_ann_list1), len(person_ann_list2) , len(person_ann_list3))

291875 18456 12392 24640


In [37]:
person_ann_list1[0]


{'segmentation': [[640.0,
   408.25,
   639.54,
   4.33,
   546.25,
   5.29,
   426.04,
   1.44,
   427.0,
   3.37,
   538.56,
   12.98,
   558.75,
   63.95,
   555.87,
   95.69,
   557.79,
   138.01,
   566.45,
   171.67,
   579.91,
   189.94,
   587.61,
   225.52,
   570.3,
   292.84,
   560.68,
   311.11,
   566.45,
   340.93,
   570.3,
   365.93,
   587.61,
   369.78,
   599.15,
   339.97,
   614.53,
   378.43,
   626.07,
   393.82,
   639.54,
   410.17],
  [405.84,
   2.4,
   438.54,
   49.53,
   454.89,
   75.49,
   454.89,
   112.04,
   450.08,
   130.31,
   458.74,
   151.47,
   467.39,
   174.55,
   394.3,
   145.7,
   398.15,
   79.34,
   392.38,
   45.68,
   330.83,
   20.68,
   310.63,
   19.72,
   276.01,
   15.87,
   249.08,
   12.98,
   231.77,
   13.94,
   218.31,
   12.98,
   208.69,
   7.21,
   206.77,
   3.37]],
 'area': 39122.63310000001,
 'iscrowd': 0,
 'image_id': 117891,
 'bbox': [206.77, 1.44, 433.23, 408.73],
 'category_id': 1,
 'id': 183030}

In [38]:
537548 in new_person_images_list


False

In [39]:
new_data_ann = []
new_data_image_list = []
new_data_images = []
for annotation in tqdm(data_df['annotations'][0]):
    if annotation['category_id'] == 1:
        if annotation['image_id'] not in new_person_images_list:
            # if annotation is person and is not in human parts annotation
            ann_segment = annotation['segmentation']
            ann_bbox = [int(annotation['bbox'][0]),int(annotation['bbox'][1]),int(annotation['bbox'][2]),int(annotation['bbox'][3])]
            ann_area = annotation['area']
            ann_image_id = annotation['image_id']
            ann_id = annotation['id']
            ann_iscrowd = annotation['iscrowd']
            ann_cat_id = annotation['category_id']
            new_data = {'segmentation': ann_segment,
                        'area': ann_area,
                        'image_id': ann_image_id,
                        'id': ann_id,
                        'category_id': ann_cat_id,
                        'bbox': ann_bbox,
                        'iscrowd': ann_iscrowd}
            new_data_ann.append(new_data)
            new_data_image_list.append(ann_image_id)
    elif annotation['category_id'] in custom_category_coco_category:
        # if annotation is not person
        ann_segment = annotation['segmentation']
        ann_bbox = [int(annotation['bbox'][0]),int(annotation['bbox'][1]),int(annotation['bbox'][2]),int(annotation['bbox'][3])]
        ann_area = annotation['area']
        ann_image_id = annotation['image_id']
        ann_id = annotation['id']
        ann_iscrowd = annotation['iscrowd']
        ann_cat_id = annotation['category_id']
        new_data = {'segmentation': ann_segment,
                    'area': ann_area,
                    'image_id': ann_image_id,
                    'id': ann_id,
                    'category_id': ann_cat_id,
                    'bbox': ann_bbox,
                    'iscrowd': ann_iscrowd}
        new_data_ann.append(new_data)
        new_data_image_list.append(ann_image_id)

100%|██████████| 291875/291875 [00:05<00:00, 57591.41it/s]


In [40]:
print(len(new_data_ann), len(set(new_data_image_list)))

78446 19582


In [41]:
new_data_images = [image for image in data_df['images'][0] if image['id'] in set(new_data_image_list)]

# take a look to "custom categories"

In [42]:
custom_categories

[{'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'},
 {'supercategory': 'animal', 'id': 17, 'name': 'cat'},
 {'supercategory': 'animal', 'id': 18, 'name': 'dog'},
 {'supercategory': 'accessory', 'id': 27, 'name': 'backpack'},
 {'supercategory': 'accessory', 'id': 31, 'name': 'handbag'},
 {'supercategory': 'accessory', 'id': 33, 'name': 'suitcase'},
 {'supercategory': 'person', 'id': 91, 'name': 'foot'},
 {'supercategory': 'person', 'id': 92, 'name': 'upleg'},
 {'supercategory': 'person', 'id': 93, 'name': 'lowleg'}]

In [43]:
# this is the list of the new categories
print(custom_categories)
print()
print(len(CUSTOM_CLASSES), "-", len(custom_categories))

cc_list = [0]*(len(CUSTOM_CLASSES))
CUSTOM_LABEL_MAP = {}
CUSTOM_ID_MAP = {}

for i, cc in enumerate(custom_categories):
    CUSTOM_ID_MAP[cc['id']] = i+1
    CUSTOM_LABEL_MAP[i+1] = cc['name']
    cc_list[i] = cc['name']

# the list of the names of the categories
print(CUSTOM_ID_MAP)
print(CUSTOM_LABEL_MAP)
print(cc_list)

[{'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'}, {'supercategory': 'animal', 'id': 17, 'name': 'cat'}, {'supercategory': 'animal', 'id': 18, 'name': 'dog'}, {'supercategory': 'accessory', 'id': 27, 'name': 'backpack'}, {'supercategory': 'accessory', 'id': 31, 'name': 'handbag'}, {'supercategory': 'accessory', 'id': 33, 'name': 'suitcase'}, {'supercategory': 'person', 'id': 91, 'name': 'foot'}, {'supercategory': 'person', 'id': 92, 'name': 'upleg'}, {'supercategory': 'person', 'id': 93, 'name': 'lowleg'}]

30 - 9
{2: 1, 17: 2, 18: 3, 27: 4, 31: 5, 33: 6, 91: 7, 92: 8, 93: 9}
{1: 'bicycle', 2: 'cat', 3: 'dog', 4: 'backpack', 5: 'handbag', 6: 'suitcase', 7: 'foot', 8: 'upleg', 9: 'lowleg'}
['bicycle', 'cat', 'dog', 'backpack', 'handbag', 'suitcase', 'foot', 'upleg', 'lowleg', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# Saving humans parts data

In [44]:

new_human_part_data = {
    'images': new_person_images, 
    'annotations': new_person_ann, 
    'categories': custom_categories
}

In [45]:
class MyEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__
new_human_part_data_json = MyEncoder().encode(new_human_part_data)

In [46]:
with open(f'{datapath}/annotations/custom_small_ann_human_parts_valid2014_v{version}.json', 'w') as f:
    json.dump(new_human_part_data_json, f)

# Saving non humans data


In [47]:

new_non_human_data = {
    'images': new_data_images, 
    'annotations': new_data_ann, 
    'categories': custom_categories
}

In [48]:
class MyEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__
new_non_human_data_json = MyEncoder().encode(new_non_human_data)

In [49]:
with open(f'{datapath}/annotations/custom_small_ann_non_human_valid2014_v{version}.json', 'w') as f:
    json.dump(new_non_human_data_json, f)

# updating indices

In [50]:
final_images = new_person_images + new_data_images
final_annotations = new_person_ann + new_data_ann

In [51]:
nil = []
for image in tqdm(final_images):
    try:
        if image["id"] not in all_images_list:
            nil.append(image["id"])
    except Exception as e:
        #print(image)
        pass
len(nil)

100%|██████████| 25566/25566 [00:00<00:00, 2491497.85it/s]


16947

In [52]:
final_annotations[0].keys()

dict_keys(['segmentation', 'area', 'image_id', 'id', 'category_id', 'bbox', 'iscrowd'])

In [53]:
for i in tqdm(range(len(final_annotations))):
    final_annotations[i]['id'] = i+1

100%|██████████| 172455/172455 [00:00<00:00, 3350745.10it/s]


In [54]:
nil = []
for image in tqdm(final_images):
    try:
        if image["id"] not in all_images_list:
            nil.append(image["id"])
    except Exception as e:
        #print(image)
        pass
len(nil)

100%|██████████| 25566/25566 [00:00<00:00, 2598671.39it/s]


16947

# saving all data

In [55]:
# IN THE version_02
new_person_data = {
    'images': final_images, 
    'annotations': final_annotations, 
    'categories': custom_categories
}

In [56]:
from json import JSONEncoder
class MyEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__
new_person_data_json = MyEncoder().encode(new_person_data)

In [57]:
with open(f'{datapath}/annotations/custom_small_ann_valid2014_v{version}.json', 'w') as f:
    json.dump(new_person_data_json, f)